Import librarries needed for AlexNet implementation

In [1]:
# Import neccessary libraries
import keras
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
import numpy as np
np.random.seed(1000)

Using TensorFlow backend.


Import Data set available in keras library for AlexNet (i.e. oxflower17)

In [2]:
from keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print('x_train shape:', x_train.shape)
print('No. of training images', x_train.shape[0])
print('No. of testing images', x_test.shape[0])

num_classes = 10
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


x_train shape: (50000, 32, 32, 3)
No. of training images 50000
No. of testing images 10000


Build AlexNet model using keras

In [3]:
# Build the AlexNet model using keras commands
model = Sequential()

# Layer 1: Convolution + Max pooling + Batch Normalization
# Input : 224x224x3, Output: 55x55x96, pooling: 27x27x96
# model.add(Conv2D(input_shape=(224,224,3), filters=96, kernel_size=(11, 11), strides=(4, 4), padding='same'))
model.add(Conv2D(input_shape=(32,32,3), filters=96, kernel_size=(3, 3), strides=(4, 4), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

# Layer 2: Convolution + Max pooling + Batch Normalization
# Input : 27x27x96, Output: 27x27x256, pooling: 13x13x256
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

# Layer 3: Convolution + Batch Normalization
# Input: 13x13x256, Output: 13x13x384
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

# Layer 4: Convolution + Batch Normalization
# Input: 13x13x384, Output: 13x13x384 
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

# Layer 5: Convolution + Max pooling + Batch Normalization
# Input: 13x13x384, Output: 13x13x384, pooling: 6x6x384
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Flatten())
model.add(Dense(4096, input_shape=(32*32*3,)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(4096))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(4096))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(10, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 8, 8, 96)          2688      
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 96)          384       
_________________________________________________________________
activation_1 (Activation)    (None, 8, 8, 96)          0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 96)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 256)         221440    
_________________________________________________________________
batch_normalization_2 (Batch (None, 4, 4, 256)         1024      
_________________________________________________________________
activation_2 (Activation)    (None, 4, 4, 256)         0         
__________

Compile the LeNet model

In [4]:
# Load the old weights (if available)
from pathlib import Path

weights_file = Path("model/alexnet_model_weights.h5")
if weights_file.is_file():
    model.load_weights(weights_file)
    print("Loaded model weights from disk")
    
model.compile(loss=keras.losses.categorical_crossentropy, 
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

Loaded model weights from disk


Train the model

In [12]:
model.fit(x_train, y_train,
          batch_size=256,
          epochs=1,
          verbose=1,
          validation_data =(x_test, y_test),
          shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/1
50000/50000 [==============================] - 668s 13ms/step - loss: 0.5185 - acc: 0.8215 - val_loss: 1.3269 - val_acc: 0.6202


Evaluate the Model

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Save the trained model as :
1. Weights : h5
2. Architecture : json

Load the saved files and evaluate the model 

In [ ]:
from keras.models import model_from_json

# serialize model to JSON
alexnet_model_json = model.to_json()
with open("model/alexnet_model.json", "w") as json_file:
    json_file.write(alexnet_model_json)

# serialize weights to HDF5
model.save_weights("model/alexnet_model_weights.h5")
print("Saved model to disk")

# load the saved model
alexnet_model_file = open('model/alexnet_model.json', 'r')
alexnet_model = alexnet_model_file.read()
alexnet_model_file.close()
alexnet_model = model_from_json(alexnet_model)

# load weights into new model
alexnet_model.load_weights("model/alexnet_model_weights.h5")
print("Loaded model from disk")
 
#evaluate the loaded model
alexnet_model.compile(loss=keras.losses.categorical_crossentropy, 
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

score = alexnet_model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Save the trained model as:
1. Weights : h5
2. Architecture : YAML

Load the saved file and evaluate accuracy.  

In [13]:
from keras.models import model_from_yaml

# serialize model to JSON
alexnet_model_yaml = model.to_yaml()
with open("model/alexnet_model.yaml", "w") as yaml_file:
    yaml_file.write(alexnet_model_yaml)

# serialize weights to HDF5
model.save_weights("model/alexnet_model_weights.h5")
print("Saved model to disk")

# load the saved model
alexnet_model_file = open('model/alexnet_model.yaml', 'r')
alexnet_model = alexnet_model_file.read()
alexnet_model_file.close()
alexnet_model = model_from_yaml(alexnet_model)

# load weights into new model
alexnet_model.load_weights("model/alexnet_model_weights.h5")
print("Loaded model from disk")
 
#evaluate the loaded model
alexnet_model.compile(loss=keras.losses.categorical_crossentropy, 
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

score = alexnet_model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Saved model to disk
Loaded model from disk
Test loss: 3.683359085083008
Test accuracy: 0.3551
